<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/longstaff_schwartz.least_square_mc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd

def longstaff_schwartz(paths, strike, discount_rate, exercise_dates):
    """
    Longstaff-Schwartz algorithm for American option pricing.

    Args:
        paths: A NumPy array of shape (number_of_paths, number_of_time_steps)
        representing the simulated asset price paths.
        strike: The strike price for the option. # Changed from strikes to strike
        discount_rate: The risk-free discount rate.
        exercise_dates: A list or array of exercise dates (timesteps).

    Returns:
        The estimated price of the American option.
    """

    n_paths = paths.shape[0]
    n_steps = paths.shape[1]

    # Initialize the continuation values and cashflows
    continuation_values = np.zeros(n_paths)
    cashflows = np.zeros(n_paths)

    # Iterate backward through the time steps
    for t in reversed(range(0, n_steps - 1)): # Iterate up to second to last timestep
        # Determine intrinsic value of option
        intrinsic_value = np.maximum(0, paths[:, t + 1] - strike) # Changed paths[t + 1] to paths[:, t + 1] and strikes to strike

        # Fit a regression model to estimate continuation value
        if np.any(intrinsic_value): # Only perform regression if there are positive intrinsic values
          regression_data = pd.DataFrame({'Price': paths[:, t], 'IntrinsicValue': intrinsic_value}) # Changed paths[t] to paths[:, t]
          regression = np.polyfit(regression_data['Price'], regression_data['IntrinsicValue'], 3)
          continuation_values = np.polyval(regression, paths[:, t]) # Changed paths[t] to paths[:, t]
        else:
            continuation_values = np.zeros(n_paths) # if intrinsic is always zero

        # Compare intrinsic value to continuation value, updating cash flows
        exercise = intrinsic_value > continuation_values
        cashflows[exercise] = intrinsic_value[exercise]

        # Discount cash flows to previous time step
        continuation_values = np.where(exercise, 0, continuation_values) # update value of continuation values

        # discount all values at each timestep
        continuation_values = continuation_values / (1+discount_rate)


        # Update cash flows
        cashflows = np.where(exercise, intrinsic_value, cashflows)

    return np.mean(cashflows)

# Example Usage (replace with your data)
np.random.seed(0)
n_paths = 10000
n_steps = 100
paths = np.random.randn(n_paths, n_steps).cumsum(axis=1)
strike = 1.0  # Example strike price # Changed strikes to strike

discount_rate = 0.05
exercise_dates = list(range(0, n_steps))

option_price = longstaff_schwartz(paths, strike, discount_rate, exercise_dates) # Changed strikes to strike
print(f"Estimated American option price: {option_price}")

Estimated American option price: 0.4120134543840609


In [5]:
# Let the underlying model be a Black-Scholes process
# dS_t / S_t = rate dt + sigma**2 dW_t, S_0 = 1.0
# with `rate = 0.1`, and volatility `sigma = 1.0`.
# Define drift and volatility functions for log(S_t)
rate = 0.1
def drift_fn(_, x):
  return rate - tf.ones_like(x) / 2.
def vol_fn(_, x):
  return tf.expand_dims(tf.ones_like(x), axis=-1)
# Use Euler scheme to propagate 100000 paths for 1 year into the future
times = np.linspace(0., 1, num=50)
num_samples = 100000
log_paths = tf.function(tff.models.euler_sampling.sample)(
        dim=1,
        drift_fn=drift_fn, volatility_fn=vol_fn,
        random_type=tff.math.random.RandomType.PSEUDO_ANTITHETIC,
        times=times, num_samples=num_samples, seed=42, time_step=0.01)
# Compute exponent to get samples of `S_t`
paths = tf.math.exp(log_paths)
# American put option price for strike 1.1 and expiry 1 (assuming actual day
# count convention and no settlement adjustment)
strike = [1.1]
exercise_times = tf.range(times.shape[-1])
discount_factors = tf.exp(-rate * times)
payoff_fn = make_basket_put_payoff(strike)
basis_fn = make_polynomial_basis(10)
least_square_mc(paths, exercise_times, payoff_fn, basis_fn,
                discount_factors=discount_factors)
# Expected value: [0.397]
# European put option price
tff.black_scholes.option_price(volatilities=[1], strikes=strikes,
                               expiries=[1], spots=[1.],
                               discount_factors=discount_factors[-1],
                               is_call_options=False,
                               dtype=tf.float64)
# Expected value: [0.379]

NameError: name 'tf' is not defined